<h1><center>Module 4: Toolboxes</center></h1>

By now you should have an understanding of how to utilize the Python window in ArcMap to handle geoprocessing tasks, however these are rather inconvenient to employ on a case to case basis. Even as standalone scripts, you would typically be required to close the mapping document, run your script through a command prompt window, and then reopen the map document to analyze your results. Rather than working in such a clunky fashion, we can instead create our own Python Toolboxes which can be run within ArcMap as our own custom geoprocessing tools. This module outlines the procedure for creating custom Python Toolboxes, installing them in your instance of ArcMap, and the process for automating these tools.

### 1.1 Toolboxes
If you have not already, go ahead and open the ArcToolbox from within an instance of ArcMap. This can be done with the red toolbox button next to our commonly used "Python Window" button on the toolbar. You should be presented with the following side panel:
<img src="figs/toolbox.JPG" width=1000px>

From here you can access all of the predefined tools, of which all of the code driven tools in previous modules are available. From these interfaces you can interact with all of the tools in a programmatic manner identical to the way they are laid out in the ArcPy documentation. A prime example is the "Add Field" tool found under the "Data Management Tools" and "Fields" sub category as shown:
<img src="figs/addfield.JPG" width=1000px>

These prompts are incredibly useful for using these tools on the fly, but can become burdensome when running multiple operations in sequence - especially when you plan to run that procedure multiples times or on a standardized basis. You can also access the ArcToolbox through the "Catalog" which is typically snapped to the right hand side of the ArcMap screen. From this view, you also have access to "My Toolboxes" where you can create your own subdirectory of Tools that you may use more frequently than others. However, we are most interested in creating our own Python Toolbox which can be accomplished by right clicking "My Toolboxes", "New", "Python Toolbox":
<img src="figs/pybox.png" width=1000px>
Once created, you can view the prewritten code for that toolbox by right clicking your new toolbox name and selecting "Edit". Typically, this will open the code in a native text editor which may not be the easiest to interact with. I would recommend locating the actual toolbox location and opening with Sublime. The toolboxes are typically saved in a somewhat secluded file location along the lines of "C:\Users\Username\AppData\Roaming\ESRI\Desktop10.7\ArcToolbox\My Toolboxes\", however you can view the saved location by once again right clicking your custom toolbox and selecting properties. Once found and opened you should be presented with a similar looking series of code blocks:
<img src="figs/pyboxpre.JPG" width=1000px>

If there are no syntactic highlights present, you may need to instruct Sublime to treat the syntax as Python code as the .pyt extension confuses the interpreter. This can be accomplished through: "View", "Syntax", "Python". As you can see there are multiple different functions and two seperate classes within this default Python Toolbox. The Toolbox class initiates a Toolbox object which houses all of your tools, and the Tool class initiates an object per Tool you would like to employ. As an example, you may consider the "Fields" toolset from earlier as the "Toolbox" object, and the "Add Field" operation would be the "Tool" object. This comes back to the concept of object oriented coding discussed in Module 1: In this case, the Toolbox object is initiated for each series of opertions you would like to conduct, and each Tool object can be tailored to the specific Feature Class being operated upon. This helps to segment the code and allow the operations to remain generally applicable to multiple different scenarios. 

### 1.2 Understanding Python Toolboxes
Most of the Toolbox class is self-explanatory and does not require much editing beyond supplying a name and updating the Tool list with as many tools as you create. The Tool class is where the real customization takes place. The "def \_\_init_\_\" method is known as the **constructor** in the Python world and houses the attributes that are inherent to that specific class. In this instance, the constructor contains the name of the Tool, its description, and determines if it can be run in the background of ArcMap. I would recommend preventing running in the background while you are still experimenting with Python Toolboxes as it can be difficult to gauge the effectiveness of your scripts if you allow it to run in the background without your monitoring its progress.

The "getParameterInfo" method is what facilitates the popup window you saw when examining the "Add Field" tool from within ArcMap. This is the method which defines each parameter required for the tool to operate, and passes that information on to the tool for the following operations. There is a multitude of different parameters you can create, and each parameter you define has its own list of parameters required to intantiate it. See [here](https://pro.arcgis.com/en/pro-app/arcpy/geoprocessing_and_python/defining-parameters-in-a-python-toolbox.htm) for the documentation page relating to parameters, and the below code block as a brief example of how you may choose to pass in the "Directional_Lines" shapefile for some of the operations we have run in the past:

In [2]:
def getParameterInfo(self):
        """Define parameter definitions"""
        param0 = arcpy.Parameter(
            displayName   = "Input Feature Class",
            name          = "in_Feature",
            datatype      = "DEFeatureClass", #DEFeatureClass represents a class where the datatype is consistent, i.e. polygons
            parameterType = "Required",
            direction     = "Input"
        )
        param1 = arcpy.Parameter(
            displayName   = "Buffer distance",
            name          = "dist",
            datatype      = "GPLong", #DEFeatureClass represents a class where the datatype is consistent, i.e. polygons
            parameterType = "Required",
            direction     = "Input"
        )
        params = [param0, param1]
        return params

The "isLicensed" method is really only utilized when you plan to make use of additional extensions required for your scripts to execute, and for the scope of these modules will not be employed. So long as this method returns True, the tool can be run within ArcMap.

The "updateParameters" and "updateMessages" methods are intended to modify the parameters based on the users input, as well as present validation errors when the user attempts to include parameters that are not expected. Specifically, the "updateParameters" method can populate additional parameters or run calculations based on the parameters, prior to the tool being initiated. For a list of the available properties that you can invoke on parameters, see [this link](https://pro.arcgis.com/en/pro-app/arcpy/geoprocessing_and_python/customizing-tool-behavior-in-a-python-toolbox.htm) for further information. An example of such a method for setting a default buffer distance for horizontal vs vertical wellbores may look like this:

In [6]:
def updateParameters(self, parameters):
# Check to see if the shapefile contains polylines (horizontal wells) or default to points (vertical wells)
    inFile = parameters[0].value
    shape = arcpy.Describe(infile).shapeType.upper()
    if shape == 'POLYLINE':
#         Set buffer distance to 460
        parameters[1].value = 460
    else:
#         Set buffer distance to 100
        parameters[1].value = 100
    return

The "updateMessages" method is intended to deliver realtime validation rule messages to the user as they populate the parameters in the tool initialization prompt. This can be a useful tool when designing Tools that you expect to distribute to a wide audience, and not just for personal use. By creating various messages for expected cases, you can help minimize the amount of confusion end users will experience when trying to implement your custom Tools. There are a number of different messages you can display with varying severity to the end user, and are listed as the Paramater Object Methods at the following [link](https://pro.arcgis.com/en/pro-app/arcpy/geoprocessing_and_python/customizing-tool-behavior-in-a-python-toolbox.htm). An example of an "updateMessages" function to notify the user that the default buffer values may or may not apply to their use case is shown below:

In [4]:
def updateMessages(self, parameters):
#     Check to see if the value is set to one of the defaults
    if parameters[1].value == 100 or parameters[1].value == 460:
        parameters[1].setWarningMessage(str("The value shown is a default value. Please ensure that this applies to your particular usecase."))
        warnings = parameters[1].hasWarning()
    return

The final default method contained in the Toolbox is the "execute" method which outlines which custom methods you would like to actually run, and what on. **Note:** you do not need to call any of the default methods listed above in the execute function. Each of the these are inherently run by the Toolbox when initiated, meaning you only need to call additional custom methods in the "execute" function. As this function is contained within the Tools object, and will be called on the parameters passed into each Tool, the target of each method will be the *Tool itself*. This is handled like so, where the API10 is a Tool that we will create (very similar to last Modules project):

In [5]:
 def execute(self, parameters, messages):
    """The source code of the tool."""
    self.API10()
    return

### 1.3 Writing your first Tool:
Within the Tool class you can create as many tool methods as you would like to include for that particular Tool. For instance, if you have a series of one-off tasks which are unrelated, you can create a series of Tools that can be invoked per that singular Toolbox. As an example, lets define a method for applying a buffer to the "Directional_Wells" layer which we can invoke at any time. Using the previously created Toolbox, define a new function below "updateMessages" titled dBuffer, like so:
<img src="figs/dbuffer.JPG" width=1000px>

Work through the procedure outlined in Module 3 in the Python Window of ArcMap to ensure that you have a firm grasp on the commands involved in applying a buffer to the "Directional_Lines" feature layer, with a specified buffer distance. This is where the value of the Python Window truly shows - testing your code prior to transcribing the process into a Python Toolbox. The more complicated your Tools become, the more difficult and time consuming it is to author numerous lines of untested code in the Toolbox, refresh your results in ArcMap, run through your parameters, then debug your errors when they inevitably hit. Save yourself some time by trial running your procedures in the Python Window. Once you have finished your trial run, remember that the main difference in working in the Toolbox vs the Python Window is **clearly establishing your Mapping Document and your workspace.** If you are still in anyway uncertain about the definition of your workspace, how to define it, or when to use relative vs absolute paths make sure to revisit Module 2. You may use the below as a framework for your dBuffer method:

In [2]:
 def dBuffer(self, parameters):
#         Read in your data and establish variable names
        data = parameters[0]
        dist = parameters[1].valueAsText
        desc = arcpy.Describe(data)
        fcName = desc.name
        # NOTE: Current is used in this case as this tool is intended to be run only when the Map is open. 
        # An absolute path would be necessary if planning to run while the map was closed.
        mxd = arcpy.mapping.MapDocument("CURRENT")
        df = mxd.activeDataFrame
        arcpy.env.overwriteOutput = True
        
        
#         Fill in the remainder using what you learned from Module 3
# ======== Key =============
#         default_dir = mxd.filePath[:-12]
#         arcpy.env.workspace = default_dir
#         arcpy.Buffer_analysis(data.value, default_dir + "\\Buffered_" + fcName + ".shp", dist + " Feet", method="GEODESIC")
#         arcpy.mapping.AddLayer(df, arcpy.mapping.Layer(default_dir + "\\Buffered_" + fcName))
# ==========================
        return 

Once you have some executable code in your toolbox, you can test your tool by saving your document and refreshing your toolbox in ArcMap. Refreshing your tool prior to running any changes you may have made is critical to ensuring that you are using the most up to date iteration of your tool. This is accomplished by simple right clicking your toolbox and selecting refresh, prior to executing your tool from within ArcMap. Once you have included your buffer tool in your execute method, you can begin testing your tool within ArcMap using the above method.  

The process outlined above can be repeated for as many Toolboxes as you would like to create, with as many Tools as you would like to include per box. **Note**: Before stringing a series of Tools together to be implemented on a single dataset, I would recommend to split each action into its own Tool and evaluate each step individually prior to full automation. This makes the debugging process significantly easier to handle when errors inevitably spring up!

### 1.4 Automating your tools
One important aspect of utilizing Python to handle geoprocessing in ArcMap is to remove the need for manual input - which means we can run these tools without ourselves being present. In order to do this, the tools need to be set to run on a schedule (particularly when the computer is not being used, such as in the early hours of the morning). If you are on a Windows computer (such as in the computer labs at CSM) you may choose to interface with the Windows Task Scheduler which can be found by searching for "Task Scheduler" in the Windows Search Bar.
<img src="figs/ts.JPG" width=1000px>

The task scheduler runs in the background of your Windows computer and handles all of the automated tasks that are set to run on a periodic basis - such as program updates. The Task Scheduler also allows for custom tasks to be scheduled to be run at specified times through the "Create Basic Task..." prompt located in the right pane. Once you have given your task a Name and Description, you will be prompted for the frequency that the task should be executed. For our testing purposes, choose "One Time", and then specify a time a few minutes from when you are setting up this Task so you have enough time to finish the setup. For this example, choose an arbitrary time as we are not going to actually schedule a Task at this time. For the "Action" choose "Start a program", and you should be presented with the following prompt:
<img src="figs/startprogram.JPG" width=500px>

Because we are intending for this Task to execute a geoprocessing tool without requiring ArcMap being open, we need to specify the absolute path for the python.exe that ArcMap utilizes. This is the same path as found in section 1.6 of Module 3, so refer back to your notes or the Module itself for that path. Once located, this goes in the "Program/script" input field as the executable. For the "Add Arguments" input field, enter the absolute path to the location of the script you wish to run contained within double quotation marks. Just like in section 1.6 o Module 3, we are first specifying the version of Python we want to utilize, followed by the .py script we intend to employ. **Note:** you cannot point to your Python Toolbox as the script you would like to execute due to the nature of how the Toolbox is structured. It is designed to be a guided toolset called from within ArcMap, not as a standalone script. In order to execute your tools outside of ArcMap, a few modifications must be made along with some .py script files.

### 1.5 Standalone Geoprocessing Scripts
Briefly explained in section 1.6 of Module 3, you can execute scripts outside of the ArcMap environment by saving your operations in a .py file utilizing absolute paths to your mapping document. One key aspect which was included in authoring the Tools in the Toolbox we created earlier were the role of parameters, and how we handled setting, updating, and passing those to the Tools. In many cases while programming, parameters are important in making sure that your tools/scripts can be run on a multitude of inputs and in a variety of scenarios. However, when working with the Task Scheduler I would argue that hardcoding your inputs and outputs in your script would be more beneficial. The main reasonsing being future modifications can be handled from within the script, meaning you only need to interact with the Scheduler once. As the goal of scheduling these Tasks is to handle repetitive operations without our input, for most cases this involves a consistent series of inputs/outputs which are conducive for hard coding.

Lets adapt the dBuffer Tool to a standalone script which we can schedule via the Task Scheduler to apply a 200 foot buffer to the "Directional_Lines" shapefile. In a real world application this would be an important task if you had scripts to download the "Directional_Lines" shapefile on a regular basis as it is updated and need to apply the buffer (this is actually updated daily by the COGCC, available [here](https://cogcc.state.co.us/data2.html#/downloads)). The first step in adapting the code from the Toolbox to a standalone script is to remove all references to the open Mapping Document and parameters. For example, from the previously included dBuffer tool we need to adapt the first five lines:

In [5]:
 def dBuffer(fc, bufferD):
    #         Read in your data and establish variable names
    mxd = arcpy.mapping.MapDocument()#absolute path to mapping document
    df = mxd.activeDataFrame
    default_dir = mxd.filePath[:-12]
    arcpy.env.workspace = default_dir
    arcpy.env.overwriteOutput = True
    # NOTE: Current is used in this case as this tool is intended to be run only when the Map is open. 
    # An absolute path would be necessary if planning to run while the map was closed.
    print("Input Feature Class: ", fc)
    print("Buffer Distance: ", bufferD)
    print("Beginning Buffer...")
    #         Fill in the remainder using what you learned from Module 3
    # ======== Key =============
    #         arcpy.Buffer_analysis(fc, default_dir + "\\Buffered_" + fc.rsplit('\\', 1)[-1], str(bufferD) + " Feet", method="GEODESIC")
    #         arcpy.mapping.AddLayer(df, arcpy.mapping.Layer(default_dir + "\\Buffered_" + fc.rsplit('\\', 1)[-1]))
    # ==========================
    print("Completed.")
    return 

As this is going to be a standalone script, I have included a few print statements which should be helpful for keeping track of your files as you go throgh additional tool setups. Make sure to copy your previous method and paste it into a new file within Sublime to make these modifications. Once you have filled in the missing code above, we still need to setup the rest of the script to actually use the dBuffer method. That would look something along these lines:

In [8]:
import arcpy

def dBuffer(fc, bufferD):
#     Code for method
    return 

fc = "absolute path to the feature class to be buffered"
bufferD = 200
dBuffer(fc, bufferD)

Saving this as a seperate .py file from your toolbox allows you to run this script independently and outside of ArcMap. You can execute the script via the following command from a Windows Command Prompt:
<pre><code>
 > C:\sw\arcPython\ArcGIS10.7\python.exe dBuffer.py
</code></pre>

Once you have saved, run, and verified the results of your dBuffer.py script you are ready to schedule a Task! Follow the steps outlined in section 1.4 above to schedule your script to be run a few minutes in the future and make sure that you see a terminal window appear with your print statements at your specified time. You have now scheduled a custom geoprocessing tool to be run without an instance of ArcMap!

### 1.6 Module 4 Project: 
With the knowledge you have gained regarding creating Python based geoprocessing tools, your task is to create a Toolbox of Python tools to accomplish the following tasks:
1. Clip all "Actual" horizontal well shapefiles to Arapahoe county
2. Generate API10 values for the horizontal wells in the aforementioned county
3. Apply a 200 foot buffer to the target horizontals
4. Intersect the target buffered horizontals with the "CO_Quarter_Sections" shapefile to add PLSS information to the wells

From this toolbox you should be able to call any of the above methods individually, and should be installed in your version of ArcMap. Additionally, you should combine the above tools into a singular, standalone geoprocessing script that you can schedule to run on a daily basis to handle the outlined geoprocessing in the order described. 

In [1]:
# ======= Key: Toolbox =============
import sys
import os
import arcpy

class Toolbox(object):
    def __init__(self):
        self.label = "Horizontal Prep"
        self.alias = "hp"

        # List of tool classes associated with this toolbox
        self.tools = [dClip, dBuffer, API10Gen, intersector]
        # ,dBuffer,API10Gen,intersector

class dClip(object):
    def __init__(self):
        """Define the tool (tool name is the name of the class)."""
        self.label = "Directional Clip"
        self.description = "This tool is intended to clip directional well sticks to a given county."
        self.canRunInBackground = False

    def getParameterInfo(self):
        """Define parameter definitions"""
        param0 = arcpy.Parameter(
            displayName   = "Directional Well Feature Class",
            name          = "in_feature",
            datatype      = "DEFeatureClass",
            parameterType = "Required",
            direction     = "Input"
            )
        param1 = arcpy.Parameter(
            displayName   = "County Feature Class",
            name          = "clip_feature",
            datatype      = "DEFeatureClass",
            parameterType = "Required",
            direction     = "Input"
        )
        param2 = arcpy.Parameter(
            displayName   = "County Name",
            name          = "county_name",
            datatype      = "GPString",
            parameterType = "Required",
            direction     = "Input"
        )
        params = [param0, param1, param2]
        return params

    def isLicensed(self):
        """Set whether tool is licensed to execute."""
        return True


    def updateParameters(self, parameters):
        return

    def updateMessages(self, parameters):
        # Check to see if the shapefile contains polylines (horizontal wells) or points (vertical wells)
        if (parameters[0].value) is not None:
            dirFc = parameters[0].value
            shape = arcpy.Describe(dirFc).shapeType.upper()
            if shape != 'POLYLINE':
                parameters[0].setWarningMessage(str("The source geometry does not contain Polylines. Please make sure that this feature contains proper geometry for this action"))
                errors = parameters[0].hasWarning()
        if (parameters[1].value) is not None:
            # Check to see if the Counties fc contains polygons
            countFc = parameters[1]
            shape = arcpy.Describe(countFc).shapeType.upper()
            if shape != 'POLYGON':
                parameters[1].setWarningMessage(str("The source geometry does not contain Polygons. Please make sure that this feature contains proper geometry for this action"))
                errors = parameters[1].hasWarning()

            # Make sure that the county name is contained in the counties shapefile
        if (parameters[2].value) is not None:
            countName = parameters[2].value
            contains = False
            with arcpy.da.SearchCursor(countFc.value, "County") as cursor:
                for row in cursor:
                    if(row[0] == countName):
                        contains = True
                    del row
                del cursor
            if contains == False:
                parameters[2].setErrorMessage(str("The provided county name is not present in the County shapefile."))
                errors = parameters[2].hasWarning()
        return

    def clipper(self, parameters):
        arcpy.env.overwriteOutput = True
        default_dir = arcpy.mapping.MapDocument("CURRENT").filePath
        default_dir = default_dir.rsplit('\\', 1)[0]
        arcpy.env.workspace = default_dir
        dirFc = parameters[0].value
        countFc = parameters[1].value
        countName = parameters[2].value
        dirFcName = arcpy.Describe(dirFc).name
        arcpy.MakeFeatureLayer_management(countFc, countName + "_Temp", "County = '" + countName + "'")
        arcpy.MakeFeatureLayer_management(dirFc, "Actual_" + dirFcName, "Dir_status = 'Actual'")
        arcpy.Clip_analysis("Actual_" + dirFcName, countName+"_Temp", arcpy.env.workspace+"\\" + countName + "_Actual_" + dirFcName + ".shp")
        return

    def execute(self, parameters, messages):
        self.clipper(parameters)
        return

class dBuffer(object):
    def __init__(self):
        """Define the tool (tool name is the name of the class)."""
        self.label = "Buffer Tool"
        self.description = ""
        self.canRunInBackground = False

    def getParameterInfo(self):
        """Define parameter definitions"""
        param0 = arcpy.Parameter(
            displayName   = "Input Feature Class",
            name          = "in_Feature",
            datatype      = "DEFeatureClass", #DEFeatureClass represents a class where the datatype is consistent, i.e. polygons
            parameterType = "Required",
            direction     = "Input"
        )
        param1 = arcpy.Parameter(
            displayName   = "Test Value",
            name          = "value",
            datatype      = "GPLong", #DEFeatureClass represents a class where the datatype is consistent, i.e. polygons
            parameterType = "Required",
            direction     = "Input"
        )
        params = [param0, param1]
        return params

    def isLicensed(self):
        """Set whether tool is licensed to execute."""
        return True
    
    def updateParameters(self, parameters):
    # Check to see if the shapefile contains polylines (horizontal wells) or points (vertical wells)
        inFile = parameters[0].value
        shape = arcpy.Describe(inFile).shapeType.upper()
        if shape == 'POLYLINE':
            parameters[1].value = 460
        else:
            parameters[1].value = 100
        return

    def updateMessages(self, parameters):
        # """Modify the messages created by internal validation for each tool
        # parameter.  This method is called after internal validation."""
        if parameters[1].value == 100 or parameters[1].value == 460:
            parameters[1].setWarningMessage(str("The value shown is a default value. Please ensure that this applies to your particular usecase."))
            errors = parameters[1].hasWarning()
        return

    def dBuffer(self, parameters):
        data = parameters[0]
        dist = parameters[1].valueAsText
        desc = arcpy.Describe(data)
        fcName = desc.name
        # NOTE: Current is used in this case as this tool is intended to be run only when the Map is open. An absolute path would be necessary if planning to run while the map was closed.
        mxd = arcpy.mapping.MapDocument("CURRENT")
        df = mxd.activeDataFrame
        default_dir = mxd.filePath[:-12]
        arcpy.env.workspace = default_dir
        arcpy.env.overwriteOutput = True
        arcpy.Buffer_analysis(data.value, default_dir + "\\Buffered_" + fcName, dist + " Feet", method="GEODESIC")
        arcpy.mapping.AddLayer(df, arcpy.mapping.Layer(default_dir + "\\Buffered_" + fcName))
        return 

    def execute(self, parameters, messages):
        """The source code of the tool."""
        self.dBuffer(parameters)
        return

class API10Gen(object):
    def __init__(self):
        """Define the tool (tool name is the name of the class)."""
        self.label = "API10Gen"
        self.description = "This tool is intended to generate 10 digit APIs for wells in Colorado (State code 05)."
        self.canRunInBackground = False

    def getParameterInfo(self):
        """Define parameter definitions"""
        param0 = arcpy.Parameter(
            displayName   = "Well Feature Class",
            name          = "in_feature",
            datatype      = "DEFeatureClass",
            parameterType = "Required",
            direction     = "Input"
            )
        param1 = arcpy.Parameter(
            displayName   = "API Attribute Name",
            name          = "api_name",
            datatype      = "GPString",
            parameterType = "Required",
            direction     = "Input"
        )
        params = [param0, param1]
        return params

    def isLicensed(self):
        """Set whether tool is licensed to execute."""
        return True


    def updateParameters(self, parameters):
        return

    def updateMessages(self, parameters):
        # Check to see if the fc contains the api column specified
        if (parameters[1].value) is not None:
            fc = parameters[0].value
            apiName = parameters[1].valueAsText
            if len(arcpy.ListFields(fc, apiName)) == 0:
                parameters[1].setWarningMessage(str("The attribute table does not conain the field specified."))
                errors = parameters[1].hasWarning()
        return

    def API10Gen(self, parameters):
        arcpy.env.overwriteOutput = True
        mxd = arcpy.mapping.MapDocument("CURRENT")
        default_dir = mxd.filePath
        default_dir = default_dir.rsplit('\\', 1)[0]
        arcpy.env.workspace = default_dir

        newFieldName = "API10"
        fc = parameters[0].value
        desc = arcpy.Describe(fc)
        fcName = desc.name
        cField = parameters[1].valueAsText
        arcpy.AddField_management(arcpy.env.workspace + "\\" + fcName, newFieldName, "TEXT", "", "", "", "Custom API10", "NON_NULLABLE")
        index = 0     
        APIs = []
        with arcpy.da.SearchCursor(fc, cField) as cursor:
            for row in cursor:
                if(len(row[0]) == 10):
                    APIs.append("05" + str(row[0][:-2]))
                else:
                    APIs.append("05" + str(row[0]))
                del row
        del cursor
        with arcpy.da.UpdateCursor(fc, newFieldName) as cursor:
            for row in cursor:
                row[0] = APIs[index]
                index += 1
                cursor.updateRow(row)
        del row
        del cursor
        return

    def execute(self, parameters, messages):
        self.API10Gen(parameters)
        return

class intersector(object):
    def __init__(self):
        """Define the tool (tool name is the name of the class)."""
        self.label = "Intersector"
        self.description = "This tool is intended to intersect buffered horizontal well sticks with PLSS feature class information."
        self.canRunInBackground = False

    def getParameterInfo(self):
        """Define parameter definitions"""
        param0 = arcpy.Parameter(
            displayName   = "Well Feature Class",
            name          = "in_feature",
            datatype      = "DEFeatureClass",
            parameterType = "Required",
            direction     = "Input"
            )
        param1 = arcpy.Parameter(
            displayName   = "PLSS Feature Class",
            name          = "intersect_with",
            datatype      = "DEFeatureClass",
            parameterType = "Required",
            direction     = "Input"
        )
        param2 = arcpy.Parameter(
            displayName   = "Cluster Tolerance",
            name          = "ct",
            datatype      = "GPString",
            parameterType = "Required",
            direction     = "Input"
        )
        param3 = arcpy.Parameter(
            displayName   = "Output Type",
            name          = "out_type",
            datatype      = "GPString",
            parameterType = "Required",
            direction     = "Input"
        )
        param3.value = 'Input'
        params = [param0, param1, param2, param3]
        return params

    def isLicensed(self):
        """Set whether tool is licensed to execute."""
        return True


    def updateParameters(self, parameters):
        return

    def updateMessages(self, parameters):
        return

    def intersector(self, parameters):
        mxd = arcpy.mapping.MapDocument("CURRENT")
        arcpy.env.overwriteOutput = True
        default_dir = mxd.filePath
        default_dir = default_dir.rsplit('\\', 1)[0]
        arcpy.env.workspace = default_dir

        in_features = [parameters[0].value, parameters[1].value]
        desc0 = arcpy.Describe(parameters[0].value)
        desc1 = arcpy.Describe(parameters[1].value)
        fc0Name = desc0.name[:-4]
        fc1Name = desc1.name[:-4]
        arcpy.Intersect_analysis(in_features,fc0Name + "_" + fc1Name, cluster_tolerance = parameters[2].valueasText , output_type=parameters[3].valueAsText)
        mxd.save()
        return

    def execute(self, parameters, messages):
        self.intersector(parameters)
        return

In [ ]:
#  ==========Key: Standalone Script =========
import arcpy
import sys
import time

def dBuffer(fc, bufferD):
    mxd = arcpy.mapping.MapDocument(r"C:\Users\Tannor - Desktop\Documents\ArcEnv\Python for ArcGis\CO_Wells\CO_Wells.mxd")
    df = mxd.activeDataFrame
    default_dir = mxd.filePath[:-12]
    arcpy.env.workspace = default_dir
    arcpy.env.overwriteOutput = True
    arcpy.Buffer_analysis(fc, default_dir + "\\Buffered_" + fc.rsplit('\\', 1)[-1], str(bufferD) + " Feet", method="GEODESIC")
    arcpy.mapping.AddLayer(df, arcpy.mapping.Layer(default_dir + "\\Buffered_" + fc.rsplit('\\', 1)[-1]))
    return 

def clipper(dirFc,countFc, countName):
    mxd = arcpy.mapping.MapDocument(r"C:\Users\Tannor - Desktop\Documents\ArcEnv\Python for ArcGis\CO_Wells\CO_Wells.mxd")
    default_dir = arcpy.mapping.MapDocument(r"C:\Users\Tannor - Desktop\Documents\ArcEnv\Python for ArcGis\CO_Wells\CO_Wells.mxd").filePath
    default_dir = default_dir.rsplit('\\', 1)[0]
    arcpy.env.workspace = default_dir
    arcpy.env.overwriteOutput = True
    dirFcName = arcpy.Describe(dirFc).name
    arcpy.MakeFeatureLayer_management(countFc, countName + "_Temp", "COUNTY = 'ARAPAHOE'")
   
    arcpy.MakeFeatureLayer_management(dirFc, "Actual_" + dirFcName, "Dir_status = 'Actual'")
    arcpy.Clip_analysis("Actual_" + dirFcName, countName+"_Temp", arcpy.env.workspace+"\\" + countName + "_Actual_" + dirFcName)
    return

def API10Gen(fc, cField):
    arcpy.env.overwriteOutput = True
    mxd = mxd = arcpy.mapping.MapDocument(r"C:\Users\Tannor - Desktop\Documents\ArcEnv\Python for ArcGis\CO_Wells\CO_Wells.mxd")
    default_dir = mxd.filePath
    default_dir = default_dir.rsplit('\\', 1)[0]
    arcpy.env.workspace = default_dir

    newFieldName = "API10"
    desc = arcpy.Describe(fc)
    fcName = desc.name
    arcpy.AddField_management(arcpy.env.workspace + "\\" + fcName, newFieldName, "TEXT", "", "", "", "Custom API10", "NON_NULLABLE")
    index = 0     
    APIs = []
    with arcpy.da.SearchCursor(fc, cField) as cursor:
        for row in cursor:
            if(len(row[0]) == 10):
                APIs.append("05" + str(row[0][:-2]))
            else:
                APIs.append("05" + str(row[0]))
            del row
    del cursor
    with arcpy.da.UpdateCursor(fc, newFieldName) as cursor:
        for row in cursor:
            row[0] = APIs[index]
            index += 1
            cursor.updateRow(row)
            del row
    del cursor
    return

def intersector(fc1, fc2, ct, ot):
    mxd = mxd = arcpy.mapping.MapDocument(r"C:\Users\Tannor - Desktop\Documents\ArcEnv\Python for ArcGis\CO_Wells\CO_Wells.mxd")
    arcpy.env.overwriteOutput = True
    default_dir = mxd.filePath
    default_dir = default_dir.rsplit('\\', 1)[0]
    arcpy.env.workspace = default_dir

    in_features = [fc1, fc2]
    desc0 = arcpy.Describe(fc1)
    desc1 = arcpy.Describe(fc2)
    fc0Name = desc0.name[:-4]
    fc1Name = desc1.name[:-4]
    arcpy.Intersect_analysis(in_features,fc0Name + "_" + fc1Name, cluster_tolerance = ct , output_type=ot)
    return

# Establish the parameters for each method
# Clip parameters:
dirFc = r'C:\Users\Tannor - Desktop\Documents\ArcEnv\Python for ArcGis\CO_Wells\Directional_Lines.shp'
countFc = r'C:\Users\Tannor - Desktop\Documents\ArcEnv\Python for ArcGis\CO_Wells\Colorado_County_Boundaries.shp'
countName = 'Arapahoe'
tstart = time.time()
print("Beginning Clip operation:")
start = time.time()
clipper(dirFc, countFc, countName)
end = time.time()
print("Clip Complete. (" + str(end-start) + ")")
# API10Gen parameters
apiFc = r'C:\Users\Tannor - Desktop\Documents\ArcEnv\Python for ArcGis\CO_Wells\Arapahoe_Actual_Directional_Lines.shp'
cField = r'API'
print("Generating API10s:")
start = time.time()
API10Gen(apiFc, cField)
end = time.time()
print("APIs generated. (" + str(end-start) + ")")
# dBuffer parameters
print("Starting Buffer:")
start = time.time()
dbFc = r"C:\Users\Tannor - Desktop\Documents\ArcEnv\Python for ArcGis\CO_Wells\Arapahoe_Actual_Directional_Lines.shp"
bufferD = 200
dBuffer(dbFc, bufferD)
end = time.time()
print("Buffer Complete. (" + str(end-start) + ")")
# Intersector parameters
print("Starting intersector:")
start = time.time()
fc1 = r"C:\Users\Tannor - Desktop\Documents\ArcEnv\Python for ArcGis\CO_Wells\Buffered_Arapahoe_Actual_Directional_Lines.shp"
fc2 = r"C:\Users\Tannor - Desktop\Documents\ArcEnv\Python for ArcGis\CO_Wells\CO_Quarter_Sections.shp"
ct = '10 Feet'
ot = 'Input'
intersector(fc1, fc2, ct, ot)
end = time.time()
print("Intersection Complete. (" + str(end-start) + ")")
tend = time.time()
print("Script Completed. (" + str(tend-tstart) + ")")
